# O intuito desse notebook e recuperar todas as imagens disponiveis  sobre pneumonia e covid e disponibilizar em um banco de dados


## bibliotecas necessárias

In [3]:
!pip install pydicom

In [9]:
from imutils import paths
import pydicom as dicom
import pandas as pd
import shutil
import cv2 
import os

##  diretorios target e source

In [10]:
def getTargetDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [46]:
dir_target_diretorio_raiz = getTargetDiretorio("MaquinaLocal")
dir_target_diretorio_dataset =r"{0}/{1}".format(dir_target_diretorio_raiz,'dataset')
dir_target_diretorio_imagem_covid =r"{0}/{1}".format(dir_target_diretorio_dataset,'covid')
dir_target_diretorio_pneumonia_normal =r"{0}/{1}".format(dir_target_diretorio_dataset,'normal')
dir_target_diretorio_pneumonia_pneumonia =r"{0}/{1}".format(dir_target_diretorio_dataset,'pneumonia')

dir_source_imagens_Covid = []

dir_source_imagens_Covid_cohen =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/images'



## diretorios pra pneumonia
dir_source_rsna_pneumonia =r'D:/Repos/rsna-pneumonia-detection-challenge'
dir_source_pneumonia_csv_detailed_class =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_detailed_class_info.csv')
dir_source_pneumonia_csv_train_labels = r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_labels.csv') 

dir_source_pneumonia_detailed_class_imagens =r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_train_images')
dir_source_pneumonia_train_labels_imagens = r"{0}/{1}".format(dir_source_rsna_pneumonia,'stage_2_test_images') 






print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)


print("source")
print(dir_source_imagens_Covid)
print(dir_source_pneumonia_detailed_class_imagens)
print(dir_source_pneumonia_train_labels_imagens)




target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia
source
D:/Repos/datasetsGitHub/covid-chestxray-dataset/images
D:/Repos/rsna-pneumonia-detection-challenge/stage_2_train_images
D:/Repos/rsna-pneumonia-detection-challenge/stage_2_test_images


In [4]:
df_csv_detailed_class = pd.read_csv(dir_source_pneumonia_csv_detailed_class)
df_csv_train_labels = pd.read_csv(dir_source_pneumonia_csv_train_labels)

print(df_csv_detailed_class.shape)
print(df_csv_train_labels.shape)


(30227, 2)
(30227, 6)


In [5]:
df_csv_detailed_class.head(5)

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [13]:
df_csv_detailed_class["class"].value_counts()

No Lung Opacity / Not Normal    11821
Lung Opacity                     9555
Normal                           8851
Name: class, dtype: int64

In [14]:
map_prognostico  = { 'Lung Opacity': 'pneumonia', 'Normal' :'normal' }

In [16]:
df_rsna = df_csv_detailed_class.loc[df_csv_detailed_class['class'].isin(['Normal','Lung Opacity'])]

In [17]:
df_rsna["prognostico"] = df_rsna['class'].map(map_prognostico)




In [43]:
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

In [45]:
quantidade = 0

for row in df_rsna.head(2)[["prognostico","patientId"]].iterrows():
    imagem_name = row[1].patientId
    prognostico = row[1].prognostico
    source_imagem = "{0}/{1}.dcm".format(dir_source_pneumonia_detailed_class_imagens,imagem_name)
    if os.path.exists(source_imagem):
        dir_target =""
        if(prognostico =="normal"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_normal,imagem_name)
        elif (prognostico =="pneumonia"):
            dir_target = "{0}/{1}.jpg".format(dir_target_diretorio_pneumonia_pneumonia,imagem_name)
        
        if not os.path.exists(dir_target):
            ds = dicom.dcmread(source_imagem)
            pixel_array_numpy = ds.pixel_array
            cv2.imwrite(dir_target, pixel_array_numpy)
            count[prognostico] +=1
            



print(count)        
print(count)    


{'normal': 1, 'pneumonia': 0, 'COVID-19': 0}


## cria os diretorios se não existirem

In [30]:
lista_diretorios_target=[]
lista_diretorios_target.append(dir_target_diretorio_raiz)
lista_diretorios_target.append(dir_target_diretorio_dataset),
lista_diretorios_target.append(dir_target_diretorio_imagem_covid),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_normal),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_pneumonia)

for item in lista_diretorios_target:
    if not os.path.exists(item):
        os.mkdir(item)
    




## variaveis do diretorio dos conjuntos de dados originais



## valida se o diretorio do git recebeu novas imagens


In [31]:
source_dir_metadata_local = r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/metadata.csv'

metadata_web_git ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'



In [32]:
df_git_covid_web  = pd.read_csv(metadata_web_git,error_bad_lines=False)

df_git_covid_local = pd.read_csv(source_dir_metadata_local,error_bad_lines=False)


print(len(df_git_covid_web))
print(len(df_git_covid_local))

if(len(df_git_covid_web) > len(df_git_covid_local)):
    print('temos {0} imagens com covid a mais no git web'.format(len(df_git_covid_web) - len(df_git_covid_local)))
elif(len(df_git_covid_web) < len(df_git_covid_local)):
    print('foram removidas {0} imagens  no git web'.format(len(df_git_covid_local) - len(df_git_covid_web)))


682
565
temos 117 imagens com covid a mais no git web


In [79]:
df_covid  = pd.read_csv(source_dir_metadata_local)

In [80]:
df_covid.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN


## populando 

## Importando nossas imagens para o banco de dados SqlLite

In [33]:
def clear_diretorio(diretorio):
    for file in os.listdir(diretorio):
        full_name =os.path.join(diretorio, file)
        
        if os.path.exists(full_name):
            os.remove(full_name) 
        
        

def get_imagens_covid(dataframe,diretorio):
        lista =[]
        for (i,row) in dataframe.iterrows():
                imagemName=row["filename"]
                image_source  = "{0}/{1}".format(diretorio,imagemName)
                lista.append(image_source)

        return lista


def get_imagens_diretorio(diretorio):
        imagePaths =[]
        imagePaths = list(paths.list_images(diretorio))
        imagePaths = [ item.replace("\\","/") for item in imagePaths]
        
        return imagePaths        


## Carregar as listas de imagens

In [34]:


print("source")
print(dir_source_imagens_Covid)
print(dir_source_pneumonia_normal)
print(dir_source_pneumonia_pneumonia)
print(dir_source_validacao_pneumonia_normal)
print(dir_source_validacao_pneumonia_validacao)
print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)



source
D:/Repos/datasetsGitHub/covid-chestxray-dataset/images


NameError: name 'dir_source_pneumonia_normal' is not defined

In [55]:


source_lista_imagens_com_Covid = []
source_lista_imagens_pneumonia_normal = []
source_lista_imagens_pneumonia_Pneumonia = []

source_lista_imagens_com_Covid = get_imagens_covid(df_covid,dir_source_imagens_Covid)
source_lista_imagens_pneumonia_normal = get_imagens_diretorio(dir_source_pneumonia_normal)
source_lista_imagens_pneumonia_Pneumonia = get_imagens_diretorio(dir_source_pneumonia_pneumonia)

print(len(source_lista_imagens_com_Covid))
print(len(source_lista_imagens_pneumonia_normal))
print(len(source_lista_imagens_pneumonia_Pneumonia))






565
234
390


['D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_477.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_478.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_479.jpeg']

## Limpar os diretorios target

In [38]:
clear_diretorio(dir_target_diretorio_imagem_covid)     
clear_diretorio(dir_target_diretorio_pneumonia_pneumonia)       
clear_diretorio(dir_target_diretorio_pneumonia_normal)        
            

In [57]:

def move_imagens_from_source_to_target(source_lista,target):
    for source_path in source_lista:
        imagemName =source_path.split('/')[-1]
        source_target= "{0}/{1}".format(target,imagemName)
    # print(source_path)
    # print(source_target)
        if(os.path.exists(source_path)):
            if not os.path.exists(source_target):
                shutil.copy2(source_path,source_target)
    
    


             
        
 
        

In [60]:
source_lista =source_lista_imagens_com_Covid
target =dir_target_diretorio_imagem_covid

move_imagens_from_source_to_target(source_lista,target)   

source_lista =source_lista_imagens_pneumonia_normal
target =dir_target_diretorio_pneumonia_normal

move_imagens_from_source_to_target(source_lista,target)   


source_lista =source_lista_imagens_pneumonia_Pneumonia
target =dir_target_diretorio_pneumonia_pneumonia

move_imagens_from_source_to_target(source_lista,target)   


## listas auxiliares
